<a href="https://colab.research.google.com/github/ayeonggg/DaconPrac/blob/main/XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
import random
import pandas as pd
import numpy as np
import os
import seaborn as sns
file_path='/content/drive/MyDrive/Colab Notebooks/building_info.csv'
df=pd.read_csv(file_path,header=None)

df.colums=['건물번호','건물유형','연면적(m2)','냉방면적(m2)','태양광용량(kW)','ESS저장용량(kWh)','PCS용량(kW)']

print(df.head())
print('\n')

pd.set_option('display.max_columns',10)
print(df.head())
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestRegressor

import warnings
warnings.filterwarnings(action='ignore')
import matplotlib as mpl
%matplotlib inline

      0     1          2         3          4             5          6
0  건물번호  건물유형    연면적(m2)  냉방면적(m2)  태양광용량(kW)  ESS저장용량(kWh)  PCS용량(kW)
1     1  건물기타   110634.0   39570.0          -             -          -
2     2  건물기타  122233.47   99000.0          -             -          -
3     3  건물기타   171243.0  113950.0         40             -          -
4     4  건물기타   74312.98  34419.62         60             -          -


      0     1          2         3          4             5          6
0  건물번호  건물유형    연면적(m2)  냉방면적(m2)  태양광용량(kW)  ESS저장용량(kWh)  PCS용량(kW)
1     1  건물기타   110634.0   39570.0          -             -          -
2     2  건물기타  122233.47   99000.0          -             -          -
3     3  건물기타   171243.0  113950.0         40             -          -
4     4  건물기타   74312.98  34419.62         60             -          -


In [33]:
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/test.csv')
building_info = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/building_info.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/test.csv')

In [42]:
train_df.head()

,num_date_time,건물번호,일시,기온(C),강수량(mm),풍속(m/s),습도(%),month,day,time
0,1_20220825 00,1,20220825 00,23.5,0.0,2.2,72,8,25,0
1,1_20220825 01,1,20220825 01,23.0,0.0,0.9,72,8,25,1
2,1_20220825 02,1,20220825 02,22.7,0.0,1.5,75,8,25,2
3,1_20220825 03,1,20220825 03,22.1,0.0,1.3,78,8,25,3
4,1_20220825 04,1,20220825 04,21.8,0.0,1.0,77,8,25,4


In [43]:
test_df.head()

,건물번호,일시,기온(C),강수량(mm),풍속(m/s),습도(%)
0,1,20220825 00,23.5,0.0,2.2,72
1,1,20220825 01,23.0,0.0,0.9,72
2,1,20220825 02,22.7,0.0,1.5,75
3,1,20220825 03,22.1,0.0,1.3,78
4,1,20220825 04,21.8,0.0,1.0,77


In [46]:
train_df.drop('num_date_time', axis = 1, inplace=True)

In [47]:
train_df.head()

,건물번호,일시,기온(C),강수량(mm),풍속(m/s),습도(%),month,day,time
0,1,20220825 00,23.5,0.0,2.2,72,8,25,0
1,1,20220825 01,23.0,0.0,0.9,72,8,25,1
2,1,20220825 02,22.7,0.0,1.5,75,8,25,2
3,1,20220825 03,22.1,0.0,1.3,78,8,25,3
4,1,20220825 04,21.8,0.0,1.0,77,8,25,4


In [7]:
building_info.head()

,건물번호,건물유형,연면적(m2),냉방면적(m2),태양광용량(kW),ESS저장용량(kWh),PCS용량(kW)
0,1,건물기타,110634.00,39570.00,-,-,-
1,2,건물기타,122233.47,99000.00,-,-,-
2,3,건물기타,171243.00,113950.00,40,-,-
3,4,건물기타,74312.98,34419.62,60,-,-
4,5,건물기타,205884.00,150000.00,-,2557,1000


In [48]:
train_df = pd.merge(train_df, building_info, on='건물번호', how='left')
test_df = pd.merge(test_df, building_info, on='건물번호', how='left')

In [49]:
len(train_df)

16800

In [50]:
train_df.isna().sum()

건물번호            0
일시              0
기온(C)           0
강수량(mm)         0
풍속(m/s)         0
습도(%)           0
month           0
day             0
time            0
건물유형            0
연면적(m2)         0
냉방면적(m2)        0
태양광용량(kW)       0
ESS저장용량(kWh)    0
PCS용량(kW)       0
dtype: int64

In [52]:
test_df.isna().sum()

건물번호            0
일시              0
기온(C)           0
강수량(mm)         0
풍속(m/s)         0
습도(%)           0
건물유형            0
연면적(m2)         0
냉방면적(m2)        0
태양광용량(kW)       0
ESS저장용량(kWh)    0
PCS용량(kW)       0
dtype: int64

In [53]:
print(len(train_df[train_df['태양광용량(kW)'] == '-']))
print(len(train_df[train_df['ESS저장용량(kWh)'] == '-']))
print(len(train_df[train_df['PCS용량(kW)'] == '-']))

10752
15960
15960


In [13]:
train_df = train_df.drop(['태양광용량(kW)', 'ESS저장용량(kWh)', 'PCS용량(kW)'], axis=1)
test_df = test_df.drop(['태양광용량(kW)', 'ESS저장용량(kWh)', 'PCS용량(kW)'], axis=1)

In [57]:
train_df['일시'] = pd.to_datetime(train_df['일시'], format='%Y%m%d %H')

# date time feature 생성
train_df['hour'] = train_df['일시'].dt.hour
train_df['day'] = train_df['일시'].dt.day
train_df['month'] = train_df['일시'].dt.month
train_df['year'] = train_df['일시'].dt.year


In [115]:

import xgboost as xgb
xgb_model = xgb.XGBRegressor()
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

In [116]:
train_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train.csv')
test_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/test.csv')

In [117]:
# 결측치 처리
train_data['강수량(mm)'].fillna(0, inplace=True)
train_data['풍속(m/s)'].fillna(train_data['풍속(m/s)'].mean(), inplace=True)
train_data['습도(%)'].fillna(train_data['습도(%)'].mean(), inplace=True)
train_data['일조(hr)'].fillna(0, inplace=True)
train_data['일사(MJ/m2)'].fillna(0, inplace=True)
test_data['강수량(mm)'].fillna(0, inplace=True)
test_data['일조(hr)'] = 0
test_data['일사(MJ/m2)'] = 0

In [118]:
# 날짜 및 시간 특성 파생
train_data['일시'] = pd.to_datetime(train_data['일시'], format='%Y%m%d %H')
test_data['일시'] = pd.to_datetime(test_data['일시'], format='%Y%m%d %H')
train_data['연'] = train_data['일시'].dt.year
train_data['월'] = train_data['일시'].dt.month
train_data['일'] = train_data['일시'].dt.day
train_data['시간'] = train_data['일시'].dt.hour
test_data['연'] = test_data['일시'].dt.year
test_data['월'] = test_data['일시'].dt.month
test_data['일'] = test_data['일시'].dt.day
test_data['시간'] = test_data['일시'].dt.hour

In [119]:
# 필요하지 않은 컬럼 제거
train_data.drop(columns=['num_date_time', '일시'], inplace=True)
test_data.drop(columns=['num_date_time', '일시'], inplace=True)

# 특성과 라벨 분리
X_train = train_data.drop(columns=['전력소비량(kWh)'])
y_train = train_data['전력소비량(kWh)']

print(X_train.shape)
print(y_train.shape)

(204000, 11)
(204000,)


In [78]:
X_train.head(101)

,num_date_time,건물번호,일시,기온(C),강수량(mm),...,일사(MJ/m2),연,월,일,시간
0,1_20220601 00,1,2022-06-01 00:00:00,18.6,0.0,...,0.0,2022,6,1,0
1,1_20220601 01,1,2022-06-01 01:00:00,18.0,0.0,...,0.0,2022,6,1,1
2,1_20220601 02,1,2022-06-01 02:00:00,17.7,0.0,...,0.0,2022,6,1,2
3,1_20220601 03,1,2022-06-01 03:00:00,16.7,0.0,...,0.0,2022,6,1,3
4,1_20220601 04,1,2022-06-01 04:00:00,18.4,0.0,...,0.0,2022,6,1,4
...,...,...,...,...,...,...,...,...,...,...,...
96,1_20220605 00,1,2022-06-05 00:00:00,24.8,0.0,...,0.0,2022,6,5,0
97,1_20220605 01,1,2022-06-05 01:00:00,24.5,0.0,...,0.0,2022,6,5,1
98,1_20220605 02,1,2022-06-05 02:00:00,23.8,0.0,...,0.0,2022,6,5,2
99,1_20220605 03,1,2022-06-05 03:00:00,23.5,0.0,...,0.0,2022,6,5,3


In [79]:
y_train.head(101)

0      1085.28
1      1047.36
2       974.88
3       953.76
4       986.40
        ...   
96      963.36
97      872.16
98      798.24
99      780.00
100     807.36
Name: 전력소비량(kWh), Length: 101, dtype: float64

In [120]:
# 데이터 정규화
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(test_data)

In [121]:
# 데이터 분할 (훈련 및 검증 세트)
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train_scaled, y_train, test_size=0.2, random_state=42)

In [122]:
import xgboost as xgb

bayes_dtrain=xgb.DMatrix(X_train_scaled, y_train)
bayes_dvalid=xgb.DMatrix(X_val_split, y_val_split)

In [123]:
# 하이퍼파라미터 그리드 정의
param_grid= {
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
    'n_estimators': [5000],
    'subsample': [0.7, 0.9],
    'colsample_bytree': [0.7, 0.9],
}

In [124]:
# 그리드 서치 객체 생성
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid,
                           cv=3, n_jobs=-1, verbose=2, scoring='neg_root_mean_squared_error')


여기서부터 결과가 안 나와서 코드는 완성 못했지만 일단 올립니다.

In [ ]:
# 최적의 하이퍼파라미터 출력
grid_search.fit(X_train_split, y_train_split)
print("Best Hyperparameters:", grid_search.best_param_)

Fitting 3 folds for each of 36 candidates, totalling 108 fits


In [ ]:
# 검증 세트에서 최적 모델의 성능 평가
y_val_pred_best = grid_search.best_estimator_.predict(X_val_split)
val_rmse_best = np.sqrt(mean_squared_error(y_val_split, y_val_pred_best))
print("Validation RMSE with Best Model:", val_rmse_best)

In [ ]:
# test 데이터에 대한 예측
y_test_pred_best = grid_search.best_estimator_.predict(X_test_scaled)
test_predictions_best = pd.DataFrame({'전력소비량(kWh)': y_test_pred_best})
test_predictions_best.to_csv('test_predictions_best.csv', index=False)